<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Projects/blob/main/Machine_Perception_with_Flowers_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Hello guys
In this Notebook, A Classifier is implemented in TensorFlow to classify images of [Flowers dataset](https://www.tensorflow.org/datasets/catalog/tf_flowers) from tensorflow, It contains 3670 images falling into 5 categories:


*   Daisy
*   Roses
*   Dandelion
*   SunFlowers
*   Tulips

you can access it from here:

```gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/```



### Import Neccessary Libraries

In [7]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
print(tf.version.VERSION)

2.9.2


In [5]:
gpu_name = tf.test.gpu_device_name()
print(f'The GPU found at: {gpu_name}')

The GPU found at: 


### Get Data

In [6]:
!gsutil cat gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/train_set.csv | head -50

gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/754296579_30a9ae018c_n.jpg,daisy
gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/dandelion/18089878729_907ed2c7cd_m.jpg,dandelion
gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/dandelion/284497199_93a01f48f6.jpg,dandelion
gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/dandelion/3554992110_81d8c9b0bd_m.jpg,dandelion
gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/4065883015_4bb6010cb7_n.jpg,daisy
gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/roses/7420699022_60fa574524_m.jpg,roses
gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/dandelion/4558536575_d43a611bd4_n.jpg,dandelion
gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/7568630428_8cf0fc16ff_n.jpg,daisy
gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/tulips/7064813645_f7f48fb527.jpg,tulips
gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/sunflowers/4933229095

As tou can see, data ""TRAIN_SET"" has a path to each Image and It's type as well.

For Example:

```
gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/10712722853_5632165b04.jpg,daisy
```

Has the Format:


```
PATH, Flower_Type
```




### Reading the Image

In [11]:
def Image_read_decoder(filename, reshape_dims):
  img = tf.io.read_file(filename) #Read the Image as a compressed String
  img = tf.image.decode_jpeg(img, channels=3) #Decode
  print(img)
  img = tf.image.convert_image_dtype(img, tf.float32)
  img = tf.image.resize(img, reshape_dims)

In [12]:
Image_read_decoder("gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/dandelion/18089878729_907ed2c7cd_m.jpg", (224, 224))

tf.Tensor(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xe2\x0cXICC_PROFILE\x00\x01\x01\x00\x00\x0cHLino\x02\x10\x00\x00mntrRGB XYZ \x07\xce\x00\x02\x00\t\x00\x06\x001\x00\x00acspMSFT\x00\x00\x00\x00IEC sRGB\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf6\xd6\x00\x01\x00\x00\x00\x00\xd3-HP  \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x11cprt\x00\x00\x01P\x00\x00\x003desc\x00\x00\x01\x84\x00\x00\x00lwtpt\x00\x00\x01\xf0\x00\x00\x00\x14bkpt\x00\x00\x02\x04\x00\x00\x00\x14rXYZ\x00\x00\x02\x18\x00\x00\x00\x14gXYZ\x00\x00\x02,\x00\x00\x00\x14bXYZ\x00\x00\x02@\x00\x00\x00\x14dmnd\x00\x00\x02T\x00\x00\x00pdmdd\x00\x00\x02\xc4\x00\x00\x00\x88vued\x00\x00\x03L\x00\x00\x00\x86view\x00\x00\x03\xd4\x00\x00\x00$lumi\x00\x00\x03\xf8\x00\x00\x00\x14meas\x00\x00\x04\x0c\x00\x00\x00$tech\x00\x00\x040\x00\x00\x00\x0crTRC